# Belegaufgabe zum Blockkurs Python
## Simulation mit vtk
Ziel ist es ein Modul zu schreiben, welches eine Windmühle simuliert. Die Windmühle soll dabei nur
aus primitiven Objekten zusammengesetzt werden und einen Rotor besitzen der sich während der
Simulation dreht. Außerdem soll eine kleine GUI geschrieben werden (mit PyQt), worüber sich die
Simulation starten lässt und sich entsprechende Parameter (Rotationsgeschwindigkeit, Höhe, etc.)
einstellen lassen! Genutzt werden sollen die in der Übung vorgestellten vtk-Objektwrapperklassen!

------
The aim is to write a module that simulates a windmill. The windmill should only be made up of primitive objects and have a rotor that rotates during the simulation. A small GUI should also be written (with PyQt) through which the simulation can be started and the relevant parameters (rotation speed, height, etc.) can be set! The vtk object wrapper classes presented in the exercise should be used!



# Todo
- [X] class for objects
- [X] assemle windmill
- [x] vtk ???
- [ ] add GUI
- [ ] add dynamics
- [ ] add buttons



In [18]:
import numpy as np
import vtk

NUMBER_OF_WING = 3
REFRESH_RATE = 100 # Hz
WINDOW_W, WINDOW_H = 800, 600

material_dict = {}

def add_material(name: str, material_properties: tuple):
    if name not in material_dict:
        material_dict[name] = material_properties
    else: 
        raise Exception("Such material already exists")
    
def get_material_list():
    return material_dict.keys()


class Structure:
    def __init__(self, material):
        self.material = material

        self.vtk_body = None

        #self.position = np.zeros(3)
        self.position = np.array([0.,0.,0.])
        self.orientation = np.eye(3)
        
        self.actor = vtk.vtkActor()


    def move_by(self, vect):
        self.position += vect
        self.update()

    
    def rotate_by(self, theta, axis='z'):
        if axis == 'z':
            rotation_matrix = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
        elif axis == 'y':
            rotation_matrix = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                                        [0, 1, 0],
                                        [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'x':
            rotation_matrix = np.matrix([[1, 0, 0],
                                        [0, np.cos(theta), -np.sin(theta)],
                                        [0, np.sin(theta), np.cos(theta)]])
        else:
            raise Exception("Axis should be either \'x\', \'y\' or \'z\'")
        
        pos_vector = self.position.reshape(3,1)  # make it vertical vector
        pos_vector = rotation_matrix @ pos_vector
        self.position = pos_vector.reshape(1,3) # make it horizontal again

        self.orientation = self.orientation * rotation_matrix

        self.update()
        
    def spin_by(self, theta, axis='z'):
        if axis == 'z':
            rotation_matrix = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
        elif axis == 'y':
            rotation_matrix = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                                        [0, 1, 0],
                                        [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'x':
            rotation_matrix = np.matrix([[1, 0, 0],
                                        [0, np.cos(theta), -np.sin(theta)],
                                        [0, np.sin(theta), np.cos(theta)]])
        else:
            raise Exception("Axis should be either \'x\', \'y\' or \'z\'")
        
        self.orientation = self.orientation * rotation_matrix

        self.update()


    def create_mapper(self):
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(self.vtk_body.GetOutputPort())
        
        self.actor.SetMapper(mapper)


    def set_edge_visibility(self, on=False):
        if on:
            self.actor.GetProperty().EdgeVisibilityOn()
            self.actor.GetProperty().SetLineWidth(2)
        else:
            self.actor.GetProperty().EdgeVisibilityOff()

        
    def set_color(self, color: tuple[float]):
        self.actor.GetProperty().SetColor(color)
        

    def set_opacity(self, opacity):
        self.actor.GetProperty().SetOpacity(opacity)


    def update(self):
        pokematrix = np.eye(4,4)
        pokematrix[:,-1][:3] = self.position
        pokematrix[:3][...,:3] = self.orientation
        vtk_poke = vtk.vtkMatrix4x4()
        for i in range(4):
            for j in range(4):
                vtk_poke.SetElement(i,j, pokematrix[i,j])
        self.actor.PokeMatrix(vtk_poke)

    @property
    def get_position(self):
        return self.position
    
    @property
    def get_orientation(self):
        return self.orientation


class Fundament(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(l)
        self.vtk_body.SetYLength(h)
        self.vtk_body.SetZLength(w)
        self.dimensions = (l,h,w)

        self.create_mapper()


class Tube(Structure):
    def __init__(self, material, r, h, res=20):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCylinderSource()
        
        self.vtk_body.SetRadius(r)
        self.vtk_body.SetHeight(h)
        self.vtk_body.SetResolution(res)
        self.dimensions = (r,h,r)
        
        self.create_mapper()


class Rotor(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(l)
        self.vtk_body.SetYLength(h)
        self.vtk_body.SetZLength(w)
        self.dimensions = (l,h,w)

        self.create_mapper()


class Wing(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(h)
        self.vtk_body.SetYLength(l)
        self.vtk_body.SetZLength(w)

        self.dimensions = (h,l,w)

        self.create_mapper()



class Fan(Structure):
    global NUMBER_OF_WING
    def __init__(self, winglist: list[Wing]):
        self.material = winglist[0].material
        self.winglist = winglist
        self.assembly = vtk.vtkAssembly()
        self.dimensions = (winglist[0].dimensions[0], winglist[0].dimensions[1] * 2, winglist[0].dimensions[2])
        self.position = np.array([0.,0.,0.])
        self.orientation = np.eye(3)

        for i in range(NUMBER_OF_WING):
          
            self.winglist[i].move_by((0, self.winglist[0].dimensions[1] * 0.5, 0))
            self.winglist[i].rotate_by(i*2*np.pi/NUMBER_OF_WING)
            self.assembly.AddPart(self.winglist[i].actor)

        self.actor = self.assembly
    
    def set_color(self, color: tuple[float]):
        for winginstance in self.winglist:
            winginstance.set_color(color)
    
    def set_opacity(self, opacity):
        for winginstance in self.winglist:
            winginstance.set_opacity(opacity)
    
    def set_edge_visibility(self, on=False):
        for winginstance in self.winglist:
            winginstance.set_edge_visibility(on)

    

class Protector(Structure):
    def __init__(self, material, r, h, res=NUMBER_OF_WING*2):
        super().__init__(material)
        self.vtk_body = vtk.vtkConeSource()
        
        self.vtk_body.SetRadius(r)
        self.vtk_body.SetHeight(h)
        self.vtk_body.SetResolution(res)
        self.dimensions = (r,h,r)
        
        self.create_mapper()


class Windmill():
    def __init__(self, fund: Fundament, tube: Tube, rotor: Rotor, fan: Fan, prot: Protector):
        self.fund = fund
        self.tube = tube
        self.rotor = rotor
        self.fan = fan
        self.prot = prot
        
        self.tick = int(1000/REFRESH_RATE)
        self.rpm = 20



    def arrange_parts(self):
        self.rotor.move_by((0, 
                           0,
                           -self.rotor.dimensions[2] * 0.5 - self.fan.dimensions[2] * 0.5))
        self.tube.move_by((0,
                          -self.tube.dimensions[1] * 0.5 - self.rotor.dimensions[1] * 0.5,
                          -self.rotor.dimensions[2] * 0.5- self.fan.dimensions[2] * 0.5))
        self.fund.move_by((0,
                          -self.tube.dimensions[1] - self.rotor.dimensions[1] * 0.5 - self.fund.dimensions[1] * 0.5,
                          -self.rotor.dimensions[2] * 0.5))
        self.prot.rotate_by(3* np.pi / 2, 'y')
        self.prot.move_by((0,
                           0,
                           self.prot.dimensions[1] * 0.5 + self.fan.dimensions[2] * 0.5))
        
    def paint_parts(self):
        self.rotor.set_color(tuple(x/255 for x in material_dict[self.rotor.material][0]))
        self.rotor.set_edge_visibility(True)

        self.tube.set_color(tuple(x/255 for x in material_dict[self.tube.material][0]))
        self.tube.set_opacity(0.1)
        self.tube.set_edge_visibility(True)

        self.fund.set_color(tuple(x/255 for x in material_dict[self.fund.material][0]))
        self.fund.set_edge_visibility(True)

        self.fan.set_color(tuple(x/255 for x in material_dict[self.fan.material][0]))
        self.fan.set_opacity(0.8)
        self.fan.set_edge_visibility(True)

        self.prot.set_color(tuple(x/255 for x in material_dict[self.prot.material][0]))
        self.prot.set_edge_visibility(True)

    def spin_all(self, angle):
        self.fan.spin_by(angle, 'z')
        #self.fund.rotate_by(angle, 'z')
        #self.tube.rotate_by(angle, 'z')
        #self.rotor.rotate_by(angle, 'z')

    def change_speed(self, *args):
        print("interaction called, new speed = %d" % (self.slider.GetRepresentation().GetValue()))
        self.rpm = self.slider.GetRepresentation().GetValue()
        self.interactor.GetRenderWindow().Render()


    
    def render_parts(self):
        global WINDOW_W, WINDOW_H

        self.rendr = vtk.vtkRenderer()
        
        self.rendrwind = vtk.vtkRenderWindow()
        self.rendrwind.AddRenderer(self.rendr)

        self.rendr.AddActor(self.tube.actor)
        self.rendr.AddActor(self.fund.actor)
        self.rendr.AddActor(self.rotor.actor)
        self.rendr.AddActor(self.fan.actor)
        self.rendr.AddActor(self.prot.actor)

        self.rendr.SetBackground(150/255, 100/255, 140/255)
        self.rendrwind.SetSize(WINDOW_W, WINDOW_H)
        self.rendrwind.SetWindowName('Windmill Simulation')

        self.textactor = self.show_text('')
        
        self.rendr.AddActor(self.textactor)

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.rendrwind)
        self.interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
        
        self.interactor.Initialize()
        self.slider = self.add_slider()
        self.interactor.AddObserver('TimerEvent', self.update_scene)

        self.interactor.CreateRepeatingTimer(20)

        self.interactor.AddObserver('InteractionEvent', self.change_speed)



    def show_text(self, text: str, fontsize = 18, color = (1,1,1)):
        txt = vtk.vtkTextActor()
        txt.SetPosition(8, 6)

        txt.GetTextProperty().SetFontSize(fontsize)
        txt.GetTextProperty().SetJustificationToLeft()
        txt.GetTextProperty().SetVerticalJustificationToBottom()
        txt.GetTextProperty().BoldOn()
        txt.GetTextProperty().SetColor(*color)
        txt.GetTextProperty().SetFontFamilyToArial()
        txt.SetInput(text)
        
        return txt
    
    def add_slider(self):
        slider_repr = vtk.vtkSliderRepresentation2D()
        slider_repr.SetMinimumValue(0.0)
        slider_repr.SetMaximumValue(100.0)
        slider_repr.SetValue(self.rpm)
        slider_repr.SetTitleText("rpm")
        slider_repr.GetSliderProperty().SetColor(.2,.2,.2)
        slider_repr.GetTitleProperty().SetColor(.2,.2,.2)
        slider_repr.GetLabelProperty().SetColor(.2,.2,.2)
        slider_repr.GetSelectedProperty().SetColor(0.31, 0.13, 0.48)
        slider_repr.GetTubeProperty().SetColor(.2,.2,.2)
        slider_repr.GetCapProperty().SetColor(.2,.2,.2)
        slider_repr.SetSliderLength(.05)
        slider_repr.SetSliderWidth(.05)
        slider_repr.SetEndCapLength(.02)

        slider_repr.GetPoint1Coordinate().SetCoordinateSystemToNormalizedDisplay()
        slider_repr.GetPoint1Coordinate().SetValue(0.9, 0.9)
        slider_repr.GetPoint2Coordinate().SetCoordinateSystemToNormalizedDisplay()
        slider_repr.GetPoint2Coordinate().SetValue(0.9, 0.1)

        widget = vtk.vtkSliderWidget()

        widget.SetInteractor(self.interactor)
        widget.SetRepresentation(slider_repr)
        #widget.SetAnimationModeToAnimate()
        widget.EnabledOn() 
        return widget

    def update_scene(self, *args):
        angle =  2* np.pi * self.tick / 1000* self.rpm / 60
    
        self.spin_all(angle)

        self.textactor.SetInput('Current speed: %.2f rpm \nCurrent tick:' % self.rpm)
        self.interactor.GetRenderWindow().Render()

# 1 rotation is 2pi rad
# we rotate self.speed / self.tick rad / s
# 1 rot / current speed = curren time for 1 rot in sec 
    def visualize(self):
        self.arrange_parts()
        self.paint_parts()
        self.render_parts()
        self.interactor.Initialize()
        
        self.interactor.GetRenderWindow().Render()
        self.interactor.Start()
        self.interactor.GetRenderWindow().Finalize()


add_material("Steel", ((145, 161, 163),))  # colour, additional parameters
add_material("Wood", ((128, 56, 1),))
add_material("Glass", ((209, 243, 255),))
add_material("Concrete", ((124, 135, 135),))
add_material("Bronze", ((190,93,4),))
add_material("Copper", ((25, 120, 85),))


Tube1 = Tube("Glass", 4, 120)
Fundament1 = Fundament("Bronze", 100, 100, 12)
Rotor1 = Rotor("Steel", 18, 18, 16)
Protector1 = Protector("Copper", 4, 6)

wings_list = [Wing("Wood", 60, 1, 4) for _ in range(NUMBER_OF_WING)]

Fan1 = Fan(wings_list)

Windmill1 = Windmill(Fundament1, Tube1, Rotor1, Fan1, Protector1)

Windmill1.visualize()



interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, new speed = 65
interaction called, 

In [3]:
r,g,b = 80,32,122
print("%.2f, %.2f, %.2f" % (r/255, g/255, b/255))

0.31, 0.13, 0.48


Class Structure:

position, orientation, material, create_vtk_body, move_by, rotate_by, move_rotate_by, create_actor, create_mapper



There are objects:

- Fundament (name, mass, width, length, material)
- Tube (name, mass, diameter, heigth, material)
- Rotor (name, mass, max_rpm, width=length, height, efficiency, model)
- Wing (name, mass, number_of_wings, material, length, width)
---
therefore 
- class Structure(name, mass, dimensions, material) can be used for everything ->
- Rotor is an ingeritage of sructure with max_rpm, model and efficiency parameter
- Wing is an ingeritage of structure with number_of_wings parameter

In [38]:
import numpy as np
import vtk

poke_matrix2 = vtk.vtkMatrix4x4()
poke_matrix = np.zeros((4,4))
position = np.matrix([1,2,3])
orientation = np.matrix([[5,6,7],
                         [6,7,8],
                         [7,8,9]])
poke_matrix[:,-1][:3] = position
poke_matrix[:3][...,:3]  = orientation

poke_matrix2 = poke_matrix
poke_matrix2


array([[5., 6., 7., 1.],
       [6., 7., 8., 2.],
       [7., 8., 9., 3.],
       [0., 0., 0., 0.]])

In [58]:
add_material("Steel", ((227, 213, 215), 600, 90))  # colour, how strong it is, how it heats
add_material("Wood", ((102, 46, 16), 100, 30))
add_material("Glass", ((209, 243, 255), 20, 40))
add_material("Concrete", ((124, 135, 135), 200, 20))

print(get_material_list())


Fundament1 = Structure("Fundament", 1000.0, "Concrete", 40, 40)
structure_list.append(Fundament1)

print(Fundament1)
    
Rotor1 = Rotor("Rotor", 420, "Steel", 1400, "mk. 1", 0.6, 8, 6)   
rotor_list.append(Rotor1)

print(Rotor1) 

Tube1 = Structure("Tube", 650.0, "Glass", 4, 60)
structure_list.append(Tube1)

print(Tube1)

Wing1 = Wing("Wing 3x", 80, "Wood", 3, 30, 4)
wing_list.append(Wing1)

print(Wing1)

dict_keys(['Steel', 'Wood', 'Glass', 'Concrete'])
This is a Structure Fundament of 1000 kg mass, made of Concrete. The size is 40x40
This is a Rotor Rotor of 420 kg mass, made of Steel. The size is 6x8
This is a Structure Tube of 650 kg mass, made of Glass. The size is 4x60
This is a Wing Wing 3x of 80 kg mass, made of Wood. The size is 4x30


In [51]:


def main():
    colors = vtkNamedColors()

    # Create a cylinder
    cylinderSource = vtkCylinderSource()
    cylinderSource.SetCenter(0.0, 0.0, 0.0)
    cylinderSource.SetRadius(5.0)
    cylinderSource.SetHeight(7.0)
    cylinderSource.SetResolution(100)

    # Create a mapper and actor
    mapper = vtkPolyDataMapper()
    mapper.SetInputConnection(cylinderSource.GetOutputPort())
    actor = vtkActor()
    actor.GetProperty().SetColor(colors.GetColor3d('Cornsilk'))
    actor.SetMapper(mapper)

    # Create a renderer, render window, and interactor
    renderer = vtkRenderer()
    renderWindow = vtkRenderWindow()
    renderWindow.SetWindowName('Cylinder')
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)

    # Add the actor to the scene
    renderer.AddActor(actor)
    renderer.SetBackground(colors.GetColor3d('DarkGreen'))

    # Render and interact
    renderWindow.Render()
    renderWindowInteractor.Start()


#if __name__ == '__main__':
#    main()

Coordiatetes are always ```x, y, z```, where x-y in horisontal and z is vertical for normal state, therefore:
- For Fundament coordinates of the center should be ```0, 0, -0.05*tube._dim[1]``` and dimensions respectively ```dim[1]//2, dim[0]//2, 0.1 * tube._dim[1]```: W, L predefined, H = .1 of tube's H.
- Then the top surface of fundament shoud be ```z=0``` and it will be centered over (0, 0) (x,y)-wise
-----
- For Tube coordinates of the center are ```(0, 0, tube._dim[1]*0.5)```. Radius is tube._dim[0]*0.5 and Height = tube._dim[1]
-----
- For Rotor coordinates of the center are ```(0, 0, tube._dim[1] + rotor.dim[0]//2)``` and dimensions rotor.dim[1]x rotor.dim[1] x rotor.dim[0] (it is square 10 x 10 and 8 in height for example)
-----
- For Fan (which is wings * number of wings) center is ```(0, rotor.dim[1]//2,  tube._dim[1] + rotor.dim[0]//2 + wing.dim[0]//2)``` -- z is the same, x is also 0, y is moved by a half thickness of rotor and a half thickness of wing. Dimensions No idea

In [65]:
import numpy as np

class Windmill():
    def __init__(self, fund: Structure, tube: Structure, rotor: Rotor, wing: Wing):
        self._fund = fund
        self._tube = tube
        self._wing = wing
        self._rotor = rotor
        self.vtk_parts_list = []
        self.vtk_actors_list = []
        self.fan_position = np.array([0.,0.,0.])
        self.fan_orientation = np.eye(3)
    

    def create_vtkCubeSource(self, struct, center, l, w, h):
        vtk_obj = vtkCubeSource()
        vtk_obj.SetCenter(center)
        vtk_obj.SetXLength(l)
        vtk_obj.SetYLength(h)
        vtk_obj.SetZLength(w)
        vtk_obj.structure = struct

        return vtk_obj
    

    # rotation axis is (0, 0, 1)
    #rotation matrix around this axis for phi rad is
    ''' 
    (cos phi + (1-cos phi) x*x) ((1-cos phi)*(xy)-sin phi z) ((1-cos phi)xz + sin phi y)
    ((1-cos phi)(yx)+sin phi z) (cos phi + (1-cos phi) y*y)) ((1-cos phi)yz + sin phi x)
    (1-cos phi)zx-sin phi y     1-cos phi zy + sin phi x       cos phi + (1-cos phi)z*z
    '''
    def turn_by(self, pos, ori, phi):

        rotation_matrix = np.matrix([[np.cos(phi), -np.sin(phi), 0],
                                     [np.sin(phi), np.cos(phi), 0],
                                     [0, 0, 1]])

        pos_new = pos * rotation_matrix       
        ori_new = ori
        return pos_new, ori_new


    def create_vtkCylinderSource(self, struct, center, r, h, resolution = 100):
        vtk_obj = vtkCylinderSource()
        vtk_obj.SetCenter(center)
        vtk_obj.SetRadius(r/2)
        vtk_obj.SetHeight(h)
        vtk_obj.SetResolution(resolution)
        vtk_obj.structure = struct

        return vtk_obj


    def assemble(self):
        tube_d, tube_h = self._tube.get_dimensions()
        fund_w, fund_l = self._fund.get_dimensions()  # W < L
        fund_h = tube_h * 0.1 
        rotor_h, rotor_w = self._rotor.get_dimensions()  # L is x, W is z, H is y, H < W = L
        rotor_l = rotor_w
        wing_w, wing_l = self._wing.get_dimensions()
        wing_h = wing_w * 0.1
        
        self._fund_center = (0.0, -0.5 * fund_h, 0.0)
        self._tube_center = (0.0, tube_h * 0.5, 0.0)
        self._rotor_center = (0.0, tube_h + rotor_h * 0.5, 0.0)
        self._fan_center = (wing_l * 0.5, tube_h + rotor_h * 0.5, rotor_w * 0.5 + wing_h * 0.5)
        
        print("fund center " + str(self._fund_center))
        print("fund l w h %f, %f, %f" % (fund_l, fund_w, fund_h))
        print("tube center " + str(self._tube_center))
        print("tube r, h %f, %f" %(tube_d/2, tube_h))
        print('rotor center' + str(self._rotor_center))
        print('fan center' + str(self._fan_center))
        self.Fundament_vtkSource = self.create_vtkCubeSource(self._fund, self._fund_center, fund_l, fund_w, fund_h)
        self.vtk_parts_list.append(self.Fundament_vtkSource)
        self.Rotor_vtkSource = self.create_vtkCubeSource(self._rotor, self._rotor_center, rotor_l, rotor_w, rotor_h)
        self.vtk_parts_list.append(self.Rotor_vtkSource)
#NEED TO CHANGE
        self.Fan_vtkSource = self.create_vtkCubeSource(self._wing, self._fan_center, wing_l, wing_h, wing_w) 
        self.vtk_parts_list.append(self.Fan_vtkSource)
#NEED TO CHANGE
        self.Tube_vtkSource = self.create_vtkCylinderSource(self._tube, self._tube_center, tube_d, tube_h, 10)
        self.vtk_parts_list.append(self.Tube_vtkSource)

        return self.vtk_parts_list
    

    def create_vtkActor(self, vtk_obj):
        mapper = vtkPolyDataMapper()
        mapper.SetInputConnection(vtk_obj.GetOutputPort())
        mapper.ScalarVisibilityOff()
        actor = vtkActor()
        actor.SetMapper(mapper)

        material = vtk_obj.structure.get_material()
        color = tuple(i/255 for i in material_dict[material][0])  # make 0-255 to 0.0-1.0

        actor.GetProperty().EdgeVisibilityOn()
        actor.GetProperty().SetLineWidth(2)

        
        actor.GetProperty().SetColor(color)
        actor.GetProperty().Modified()
        
        print(material + str(actor.GetProperty().GetColor()))
        
        return actor
    

    def initiate_all_actors(self):
        self.vtk_actors_list = []
        for vtk_part in self.vtk_parts_list:
            actor = self.create_vtkActor(vtk_part)
            self.vtk_actors_list.append(actor)

        return self.vtk_actors_list


    def render(self, lst):
        # Create a renderer, render window, and interactor
        renderer = vtkRenderer()
        renderWindow = vtkRenderWindow()
        renderWindow.AddRenderer(renderer)
        renderWindowInteractor = vtkRenderWindowInteractor()
        renderWindowInteractor.SetRenderWindow(renderWindow)
                
        for actor in lst:
            renderer.AddActor(actor)
                
        renderer.SetBackground(.8,.8,.8)
        renderWindow.Render()
        renderWindowInteractor.Start()
        
        pass

    def change_rotation_speed(self):
        pass

    def change_part(self):
        pass


if __name__ is '__main__':
    Windmill1 = Windmill(Fundament1, Tube1, Rotor1, Wing1)

    Windmill1.assemble()


    list_of_actors = Windmill1.initiate_all_actors()
    Windmill1.render(list_of_actors)


<>:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\user\AppData\Local\Temp\ipykernel_15736\455553112.py:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if __name__ is '__main__':


fund center (0.0, -3.0, 0.0)
fund l w h 40.000000, 40.000000, 6.000000
tube center (0.0, 30.0, 0.0)
tube r, h 2.000000, 60.000000
rotor center(0.0, 63.0, 0.0)
fan center(15.0, 63.0, 4.2)
Concrete(0.48627450980392156, 0.5294117647058824, 0.5294117647058824)
Steel(0.8901960784313725, 0.8352941176470589, 0.8431372549019608)
Wood(0.4, 0.1803921568627451, 0.06274509803921569)
Glass(0.8196078431372549, 0.9529411764705882, 1.0)


In [26]:
import vtk
import numpy as np
'''
[t00 t01 t02 x]
[t10 t11 t12 y]
[t20 t21 t22 z]
[0   0   0   0]

t is 3x3 matrix of the orientation

x y z are the position

we move wing in rotate wing 
'''

r = np.array([1.2,1.2,1.2])
T = np.eye(3)
source = vtk.vtkCubeSource()
actor = vtk.vtkActor()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(source.GetOutputPort())

source.SetXLength(2)
source.SetYLength(2)
source.SetZLength(2)

actor.SetMapper(mapper)

actor.GetProperty().SetColor(.3,.4,.5)



source2 = vtk.vtkCubeSource()
actor2 = vtk.vtkActor()
mapper2 = vtk.vtkPolyDataMapper()
mapper2.SetInputConnection(source2.GetOutputPort())

source2.SetXLength(2)
source2.SetYLength(2)
source2.SetZLength(2)
source2.SetCenter(3,0,0)

actor2.SetMapper(mapper2)

actor2.GetProperty().SetColor(.5,.4,.3)



def update(actor1):
    global r, T
    poke = vtk.vtkMatrix4x4()

# Matrix elementweise befuellen
    for row in range(3):

    # Positionsvektor
        poke.SetElement(row,3, r[row])

    # Orientierung
        for col in range(3):
            poke.SetElement(row, col, T[row,col])

# Matrix an Actor übergeben
    actor1.PokeMatrix(poke)
    print(poke)

rendr = vtk.vtkRenderer()
rendrwind = vtk.vtkRenderWindow()
rendrwind.AddRenderer(rendr)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(rendrwind)
interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
interactor.Initialize()

update(actor)

rendr.AddActor(actor)
rendr.AddActor(actor2)

slider = vtk.vtkSliderRepresentation2D()
slider.SetMinimumValue(0.0)
slider.SetMaximumValue(1000.0)
slider.SetValue(20.0)
slider.SetTitleText("rpm")
slider.GetSliderProperty().SetColor(.2,.2,.2)
slider.GetTitleProperty().SetColor(.2,.2,.2)
slider.GetLabelProperty().SetColor(.2,.2,.2)
slider.GetSelectedProperty().SetColor(.2,.2,.2)
slider.GetTubeProperty().SetColor(.2,.2,.2)
slider.GetCapProperty().SetColor(.2,.2,.2)
slider.SetSliderLength(.05)
slider.SetSliderWidth(.05)
slider.SetEndCapLength(.02)

slider.GetPoint1Coordinate().SetCoordinateSystemToNormalizedDisplay()
slider.GetPoint1Coordinate().SetValue(0.2, 0.1)
slider.GetPoint2Coordinate().SetCoordinateSystemToNormalizedDisplay()
slider.GetPoint2Coordinate().SetValue(0.8, 0.1)

widget = vtk.vtkSliderWidget()
widget.SetInteractor(interactor)
widget.SetRepresentation(slider)
widget.EnabledOn() 
#self.rendr.AddActor(self.fan.actor)
rendr.SetBackground(150/255, 100/255, 140/255)
rendrwind.SetSize(600, 600)
widget.AddObserver('InteractionEvent', update)
interactor.Initialize()
rendrwind.Render()
interactor.Start()


#self.interactor.AddObserver('TimerEvent', self.update_scene())
#self.interactor.CreateRepeatingTimer(100) # 100 ms
rendrwind.Render()
interactor.Start()





vtkMatrix4x4 (0000025CCF7CBAA0)
  Debug: Off
  Modified Time: 26837622
  Reference Count: 3
  Registered Events: (none)
  Elements:
    1 0 0 1.2 
    0 1 0 1.2 
    0 0 1 1.2 
    0 0 0 1 




TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

...

TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

TypeError: update() takes 1 positional argument but 2 were given

: 